In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime

import numpy as np
from operator import itemgetter

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%load_ext autoreload
%autoreload 2

In [6]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM
from libs.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [7]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
ICM_train, ICM_validation = split_train_in_two_percentage_global_sample(ICM_all, train_percentage = 0.80)

In [8]:
from libs.Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 157 ( 0.4%) Users that have less than 1 test interactions


In [17]:
from libs.Recommenders.KNN.ItemKNNCBFCFSimilarityHybridRecommender import ItemKNNCBFCFSimilarityHybridRecommender

In [11]:
results = pd.DataFrame()

In [12]:
save_path = "result_experiments/ItemKNNCBFCFSimilarityHybridRecommender/"

In [15]:
if os.path.exists(save_path+"results.csv"):
    results = pd.read_csv(save_path+"results.csv")
else:
    os.makedirs(save_path)

In [13]:
max_MAP = -1 

In [ ]:
for topK_hybrid in [25, 50, 100, 300]:
    for alpha in np.linspace(0.05, 0.95, 10):
        for topK_knncf in [25, 50, 100, 300]:
            for shrink_knncf in [50, 100, 150]:
                for feature_weighting_knncf in ["none", "BM25", "TF-IDF"]:
                    for topK_knncbf in [25, 50, 100, 300]:
                        for shrink_knncbf in [50, 100, 150]:
                            for feature_weighting_knncbf in ["none", "BM25", "TF-IDF"]:
        
                                current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                                params = {
                                    "alpha": alpha, 
                                    "topK_hubrid": topK_hybrid,
                                    "topK_knncf": topK_knncf,
                                    "shrink_knncf": shrink_knncf,
                                    "feature_weighting_knncf": feature_weighting_knncf,
                                    "topK_knncbf": topK_knncbf,
                                    "shrink_knncbf": shrink_knncbf,
                                    "feature_weighting_knncbf": feature_weighting_knncbf,
                                    "time": current_timestamp
                                }
                                
                                model_knn = ItemKNNCBFCFSimilarityHybridRecommender(
                                    alpha = alpha,
                                    topK_hybrid=topK_hybrid,
                                    URM_train = URM_train,
                                    ICM_train = ICM_train,
                                    topK_knncf=topK_knncf,
                                    shrink_knncf=shrink_knncf,
                                    feature_weighting_knncf=feature_weighting_knncf,
                                    topK_knncbf=topK_knncbf,
                                    shrink_knncbf=shrink_knncbf,
                                    feature_weighting_knncbf=feature_weighting_knncbf,
                                )
        
                                metrics_df, _ = evaluator_validation.evaluateRecommender(model_knn)
                                if max_MAP < metrics_df["MAP"][10]:
                                    max_MAP = metrics_df["MAP"][10]
                                print(', '.join(f"{key}: {value}" for key, value in params.items()), "MAP: ", metrics_df["MAP"][10], "max MAP: ", max_MAP)
                                      
                                params_df = pd.DataFrame([params])
                                iteration_results = pd.concat([params_df, metrics_df.reset_index()], axis=1)
                                results = pd.concat([results, iteration_results], ignore_index=True)
                                results.to_csv(save_path+"results.csv", index=False)

Similarity column 38121 (100.0%), 4840.42 column/sec. Elapsed time 7.88 sec
ItemKNNCBFRecommender: ICM Detected 42 ( 0.1%) items with no features.
Similarity column 38121 (100.0%), 1514.68 column/sec. Elapsed time 25.17 sec
EvaluatorHoldout: Processed 35579 (100.0%) in 9.79 sec. Users per second: 3635
alpha: 0.05, topK_hubrid: 25, topK_knncf: 25, shrink_knncf: 50, feature_weighting_knncf: none, topK_knncbf: 25, shrink_knncbf: 50, feature_weighting_knncbf: none, time: 2024-11-28 12:45:44 MAP:  0.013427230125493564 max MAP:  0.013427230125493564
Similarity column 38121 (100.0%), 4932.23 column/sec. Elapsed time 7.73 sec
ItemKNNCBFRecommender: ICM Detected 42 ( 0.1%) items with no features.
Similarity column 38121 (100.0%), 1508.67 column/sec. Elapsed time 25.27 sec
EvaluatorHoldout: Processed 35579 (100.0%) in 10.06 sec. Users per second: 3538
alpha: 0.05, topK_hubrid: 25, topK_knncf: 25, shrink_knncf: 50, feature_weighting_knncf: none, topK_knncbf: 25, shrink_knncbf: 50, feature_weighti